In [ ]:
import os
import sys
from pathlib import Path

# Navigate to project root (equivalent to cd ..)
project_dir = Path(__file__).parent.parent if '__file__' in globals() else Path.cwd().parent
os.chdir(project_dir)

# Add src directory to Python path for imports
src_dir = project_dir / "src"
if str(src_dir) not in sys.path:
    sys.path.insert(0, str(src_dir))

# Set environment for dev testing
os.environ['REPORT_ENV'] = 'dev'

In [ ]:
import src.config

In [ ]:
import pandas as pd
from deltalake import DeltaTable
from pathlib import Path

In [ ]:
df = DeltaTable(src.config.SILVER / "account").to_pandas()

In [ ]:
import cdutils.acct_file_creation.core
from datetime import datetime

# # Specific date
specified_date = datetime(2020, 12, 31)
df = cdutils.acct_file_creation.core.query_df_on_date(specified_date)

In [ ]:
df

In [ ]:
df['mjaccttypcd'].unique()

In [ ]:
df = df[df['mjaccttypcd'].isin(['CML','MLN','CNS','MTG','CK','SAV','TD'])].copy()

In [ ]:
# Create Account Type mapping - Easier to understand, based on our major field
def map_account_type(acct_code:str):
    """
    Map mjaccttypcd to friendly Account Type
    """
    mapping = {
        'CML':'Loan',
        'MLN':'Loan',
        'CNS':'Loan',
        'MTG':'Loan',
        'CK':'Deposit',
        'SAV':'Deposit',
        'TD':'Deposit'
    }
    return mapping.get(str(acct_code).upper(), 'Other')

df['Account Type'] = df['mjaccttypcd'].apply(map_account_type)

In [ ]:
df['branchname'].unique()

In [ ]:
df

In [ ]:
region_map = {
    # ——— Attleboro/Taunton ———
    'BCSB - MUNI MAIN OFFICE': 'Attleboro/Taunton',
    'BCSB - MAIN OFFICE': 'Attleboro/Taunton',
    "BCSB - COMM'L LENDING- TAUNTON": 'Attleboro/Taunton',
    'BCSB - MUNI ATTLEBORO BRANCH': 'Attleboro/Taunton',
    'BCSB - DEPOSIT OPERATIONS': 'Attleboro/Taunton',
    'BCSB - NO ATTLEBORO BRANCH': 'Attleboro/Taunton',
    'BRISTOL COUNTY SAVINGS BANK': 'Attleboro/Taunton',
    "BCSB - COMM'L LENDING - ATTLEBORO": 'Attleboro/Taunton',
    'BCSB - BEACON SECURITY CORP': 'Attleboro/Taunton',
    'BCSB - ATTLEBORO BRANCH': 'Attleboro/Taunton',
    'BCSB - MUNI COUNTY STREET BRANCH': 'Attleboro/Taunton',
    'BCSB - REHOBOTH BRANCH': 'Attleboro/Taunton',
    'BCSB - MUNI REHOBOTH BRANCH': 'Attleboro/Taunton',
    'BCSB - MUNI NO ATTLEBORO BRANCH': 'Attleboro/Taunton',
    'BCSB - MUNI RAYNHAM CENTER BRANCH': 'Attleboro/Taunton',
    'BCSB - COUNTY STREET BRANCH': 'Attleboro/Taunton',
    'BCSB - NORTH RAYNHAM BRANCH': 'Attleboro/Taunton',
    'BCSB - RAYNHAM CENTER BRANCH': 'Attleboro/Taunton',
    "BCSB - COMM'L LENDING - FRANKLIN": 'Attleboro/Taunton',
    'BCSB - FRANKLIN BRANCH': 'Attleboro/Taunton',
    'BCSB - CONS INST LENDING- TAUNTON': 'Attleboro/Taunton',
    'BCSB - RESIDENTIAL MTG - ATTLEBORO': 'Attleboro/Taunton',
    'BCSB - RESIDENTIAL MTG- TAUNTON': 'Attleboro/Taunton',
    'BCSB - RESIDENTIAL MTG - FRANKLIN': 'Attleboro/Taunton',
    'BCSB - CONS INST LENDING - ATTLEBORO': 'Attleboro/Taunton',
    'BCSB - SMALL BUSINESS LOAN CENTER': 'Attleboro/Taunton',
    'BCSB - CONTACT CENTER': 'Attleboro/Taunton',
    'BCSB - TAUNTON HIGH SCHOOL': 'Attleboro/Taunton',
    'BCSB - MUNI ATTLEBORO HIGH SCHOOL': 'Attleboro/Taunton',
    'BCSB - ATTLEBORO HIGH SCHOOL': 'Attleboro/Taunton',
    'BCSB - INDIRECT LENDING': 'Attleboro/Taunton',

    # ——— South Coast ———
    'BCSB - MUNI FALL RIVER BRANCH': 'South Coast',
    "BCSB - COMM'L LENDING - FALL RIVER": 'South Coast',
    "BCSB - COMM'L LENDING - CANDLEWORKS": 'South Coast',
    "BCSB - COMM'L LENDING - DARTMOUTH": 'South Coast',
    'BCSB - MUNI DARTMOUTH BRANCH': 'South Coast',
    'BCSB - MUNI NB ASHLEY BLVD BRANCH': 'South Coast',
    'BCSB - NB ASHLEY BLVD BRANCH': 'South Coast',
    'BCSB - MUNI CANDLEWORKS BRANCH': 'South Coast',
    'BCSB - MUNI EAST FREETOWN BRANCH': 'South Coast',
    'BCSB - DARTMOUTH BRANCH': 'South Coast',
    'BCSB - EAST FREETOWN BRANCH': 'South Coast',
    'BCSB - FALL RIVER BRANCH': 'South Coast',
    'BCSB - CANDLEWORKS BRANCH': 'South Coast',
    'BCSB - RESIDENTIAL MTG - DARTMOUTH': 'South Coast',
    'BCSB - RESIDENTIAL MTG - FALL RIVER': 'South Coast',
    'BCSB - RESI LENDING - NEW BEDFORD': 'South Coast',

    # ——— Rhode Island ———
    "BCSB - COMM'L LENDING - WARWICK": 'Rhode Island',
    "BCSB - COMM'L LENDING - PROVIDENCE": 'Rhode Island',
    "BCSB - COMM'L LENDING - PAWTUCKET": 'Rhode Island',
    'BCSB - CUMBERLAND': 'Rhode Island',
    'BCSB - PAWTUCKET BRANCH': 'Rhode Island',
    "BCSB - CMM'L LENDING - FNB-RI": 'Rhode Island',
    'BCSB - MUNI PAWTUCKET BRANCH': 'Rhode Island',
    'BCSB - RESIDENTIAL MTG - PAWTUCKET': 'Rhode Island',
    'BCSB - MUNI GREENVILLE': 'Rhode Island',
    'BCSB - GREENVILLE': 'Rhode Island',
    'BCSB - RESI LENDING - WARWICK': 'Rhode Island',
    'BCSB - CONS INST LENDING - PAWTUCKET': 'Rhode Island',
    'BCSB - CONS INST LENDING - FNB-RI': 'Rhode Island',
    'BCSB - MUNI CUMBERLAND': 'Rhode Island',
    'BCSB - RESIDENTIAL MTG - FNB-RI': 'Rhode Island',

    # ——— Other ———
    'BCSB - RESIDENTIAL MTG - CAPE COD': 'Other',
    # Operational catch-alls (if any are left unmapped in future, they'll fall to 'Other' via the fillna below)
}

In [ ]:
# region_map = {
#     # ——— Attleboro/Taunton ———
#     'BCSB - MUNI MAIN OFFICE': 'Attleboro/Taunton',
#     'BCSB - MAIN OFFICE': 'Attleboro/Taunton',
#     "BCSB - COMM'L LENDING- TAUNTON": 'Attleboro/Taunton',
#     'BCSB - MUNI ATTLEBORO BRANCH': 'Attleboro/Taunton',
#     'BCSB - DEPOSIT OPERATIONS': 'Attleboro/Taunton',
#     'BCSB - NO ATTLEBORO BRANCH': 'Attleboro/Taunton',
#     'BRISTOL COUNTY SAVINGS BANK': 'Attleboro/Taunton',
#     "BCSB - COMM'L LENDING - ATTLEBORO": 'Attleboro/Taunton',
#     'BCSB - BEACON SECURITY CORP': 'Attleboro/Taunton',
#     'BCSB - ATTLEBORO BRANCH': 'Attleboro/Taunton',
#     'BCSB - MUNI COUNTY STREET BRANCH': 'Attleboro/Taunton',
#     'BCSB - REHOBOTH BRANCH': 'Attleboro/Taunton',
#     'BCSB - MUNI REHOBOTH BRANCH': 'Attleboro/Taunton',
#     'BCSB - MUNI NO ATTLEBORO BRANCH': 'Attleboro/Taunton',
#     'BCSB - MUNI RAYNHAM CENTER BRANCH': 'Attleboro/Taunton',
#     'BCSB - COUNTY STREET BRANCH': 'Attleboro/Taunton',
#     'BCSB - NORTH RAYNHAM BRANCH': 'Attleboro/Taunton',
#     'BCSB - RAYNHAM CENTER BRANCH': 'Attleboro/Taunton',
#     "BCSB - COMM'L LENDING - FRANKLIN": 'Attleboro/Taunton',
#     'BCSB - FRANKLIN BRANCH': 'Attleboro/Taunton',
#     'BCSB - CONS INST LENDING- TAUNTON': 'Attleboro/Taunton',
#     'BCSB - RESIDENTIAL MTG - ATTLEBORO': 'Attleboro/Taunton',
#     'BCSB - RESIDENTIAL MTG- TAUNTON': 'Attleboro/Taunton',
#     'BCSB - RESIDENTIAL MTG - FRANKLIN': 'Attleboro/Taunton',
#     'BCSB - CONS INST LENDING - ATTLEBORO': 'Attleboro/Taunton',
#     'BCSB - SMALL BUSINESS LOAN CENTER': 'Attleboro/Taunton',
#     'BCSB - CONTACT CENTER': 'Attleboro/Taunton',
#     'BCSB - TAUNTON HIGH SCHOOL': 'Attleboro/Taunton',
#     'BCSB - MUNI ATTLEBORO HIGH SCHOOL': 'Attleboro/Taunton',
#     'BCSB - ATTLEBORO HIGH SCHOOL': 'Attleboro/Taunton',
#     'BCSB - INDIRECT LENDING': 'Attleboro/Taunton',

#     # ——— South Coast ———
#     'BCSB - MUNI FALL RIVER BRANCH': 'South Coast',
#     "BCSB - COMM'L LENDING - FALL RIVER": 'South Coast',
#     "BCSB - COMM'L LENDING - CANDLEWORKS": 'South Coast',
#     "BCSB - COMM'L LENDING - DARTMOUTH": 'South Coast',
#     'BCSB - MUNI DARTMOUTH BRANCH': 'South Coast',
#     'BCSB - MUNI NB ASHLEY BLVD BRANCH': 'South Coast',
#     'BCSB - NB ASHLEY BLVD BRANCH': 'South Coast',
#     'BCSB - MUNI CANDLEWORKS BRANCH': 'South Coast',
#     'BCSB - MUNI EAST FREETOWN BRANCH': 'South Coast',
#     'BCSB - DARTMOUTH BRANCH': 'South Coast',
#     'BCSB - EAST FREETOWN BRANCH': 'South Coast',
#     'BCSB - FALL RIVER BRANCH': 'South Coast',
#     'BCSB - CANDLEWORKS BRANCH': 'South Coast',
#     'BCSB - RESIDENTIAL MTG - DARTMOUTH': 'South Coast',
#     'BCSB - RESIDENTIAL MTG - FALL RIVER': 'South Coast',
#     'BCSB - RESI LENDING - NEW BEDFORD': 'South Coast',

#     # ——— Rhode Island ———
#     "BCSB - COMM'L LENDING - WARWICK": 'Rhode Island',
#     "BCSB - COMM'L LENDING - PROVIDENCE": 'Rhode Island',
#     "BCSB - COMM'L LENDING - PAWTUCKET": 'Rhode Island',
#     'BCSB - CUMBERLAND': 'Rhode Island',
#     'BCSB - PAWTUCKET BRANCH': 'Rhode Island',
#     "BCSB - CMM'L LENDING - FNB-RI": 'Rhode Island',
#     'BCSB - MUNI PAWTUCKET BRANCH': 'Rhode Island',
#     'BCSB - RESIDENTIAL MTG - PAWTUCKET': 'Rhode Island',
#     'BCSB - MUNI GREENVILLE': 'Rhode Island',
#     'BCSB - GREENVILLE': 'Rhode Island',
#     'BCSB - RESI LENDING - WARWICK': 'Rhode Island',
#     'BCSB - CONS INST LENDING - PAWTUCKET': 'Rhode Island',
#     'BCSB - CONS INST LENDING - FNB-RI': 'Rhode Island',
#     'BCSB - MUNI CUMBERLAND': 'Rhode Island',
#     'BCSB - RESIDENTIAL MTG - FNB-RI': 'Rhode Island',

#     # ——— Other ———
#     'BCSB - RESIDENTIAL MTG - CAPE COD': 'Other',
#     # Operational catch-alls (if any are left unmapped in future, they'll fall to 'Other' via the fillna below)
# }

# # Create the Region column from the mapping
# import numpy as np
# df['Region'] = df['branchname'].map(region_map).fillna(
#     np.where(df['branchname'].str.contains(r'Warwick|Providence|Pawtucket|Cumberland|Greenville|FNB-RI', case=False), 'Rhode Island',
#     np.where(df['branchname'].str.contains(r'Fall River|Dartmouth|East Freetown|New Bedford|Candleworks|Ashley Blvd', case=False), 'South Coast',
#     np.where(df['branchname'].str.contains(r'Attleboro|Franklin|Raynham|Taunton|Rehoboth|County Street|Main Office', case=False), 'Attleboro/Taunton', 'Other'))))



In [ ]:
import pandas as pd
import numpy as np

# Assume you already have: region_map (dict) and df['Branch'] exists
s = df['branchname']

# 1) Exact-name mapping first
region = s.map(region_map)

# 2) Regex-based geographic fallback as a Series
ri = s.str.contains(r'Warwick|Providence|Pawtucket|Cumberland|Greenville|FNB-RI', case=False, na=False)
sc = s.str.contains(r'Fall River|Dartmouth|East Freetown|New Bedford|Candleworks|Ashley Blvd', case=False, na=False)
at = s.str.contains(r'Attleboro|Franklin|Raynham|Taunton|Rehoboth|County Street|Main Office', case=False, na=False)

fallback = pd.Series(
    np.select([ri, sc, at], ['Rhode Island', 'South Coast', 'Attleboro/Taunton'], default='Other'),
    index=df.index
)

# 3) Fill unmapped with the Series (allowed), not an ndarray
df['Region'] = region.fillna(fallback)


In [ ]:
df

In [ ]:
# Performed 1 aggregation grouped on columns: 'Region', 'Account Type'
grouped_df = df.groupby(['Region', 'Account Type']).agg(NetBalance_sum=('Net Balance', 'sum')).reset_index()

In [ ]:
grouped_df